In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv(r"D:\Data\kaggle\GooAvePre\11.23\train_96.csv")
test_df = pd.read_csv(r"D:\Data\kaggle\GooAvePre\11.23\test_96.csv")

C:\Users\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(train_df.shape)
print(test_df.shape)

(1708337, 96)
(401589, 96)


# train

In [11]:
train_y = train_df['totals.transactionRevenue']
train_x = train_df.drop(labels=['fullVisitorId', 'totals.transactionRevenue'],axis=1)

In [12]:
print(train_y.shape)
print(train_x.shape)

(1708337,)
(1708337, 94)


In [13]:
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.15, random_state=10)

In [14]:
lgb_train_data = lgb.Dataset(X_train, label=y_train)
lgb_val_data = lgb.Dataset(X_val, label=y_val)

In [15]:
params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 40,
        "learning_rate" : 0.005,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "seed": 42}

In [16]:
model = lgb.train(params, lgb_train_data, 
                      num_boost_round=5000,
                      valid_sets=[lgb_train_data, lgb_val_data],
                      early_stopping_rounds=100,
                      verbose_eval=500)

Training until validation scores don't improve for 100 rounds.
[500]	training's rmse: 0.000804077	valid_1's rmse: 0.00113661
[1000]	training's rmse: 0.000699057	valid_1's rmse: 0.00109448
[1500]	training's rmse: 0.000648977	valid_1's rmse: 0.00108361
Early stopping, best iteration is:
[1840]	training's rmse: 0.000624975	valid_1's rmse: 0.00108123


In [17]:
model.save_model(r'D:\Data\kaggle\GooAvePre\11.23\lgb_1123.txt')

In [18]:
y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)
y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
print(f"LGBM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train))}")

LGBM: RMSE val: 0.0010812280518348524  - RMSE train: 0.0006249753070378031


# test

In [19]:
test_df.shape

(401589, 96)

In [20]:
test_y = test_df['totals.transactionRevenue']
test_x = test_df.drop(labels=['fullVisitorId', 'totals.transactionRevenue'],axis=1)

In [21]:
pred_y = model.predict(test_x, num_iteration=model.best_iteration)

In [22]:
pred_y.shape

(401589,)

In [23]:
pred_y[:10]                         

array([ -4.11900932e-03,  -2.95237083e-07,  -3.44411418e-07,
         3.32434654e-03,  -6.88133409e-03,  -3.44411418e-07,
         1.06683983e-03,  -4.27816116e-07,  -7.94959171e-03,
        -2.57615036e-07])

In [24]:
np.sqrt(mean_squared_error(test_y, pred_y))

0.38712913311700803

# submit

In [25]:
test_df.fullVisitorId.shape

(401589,)

In [26]:
pred_y.shape

(401589,)

In [27]:
fullVisit = test_df.fullVisitorId.as_matrix().astype('str')

C:\Users\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [47]:
data = {'fullVisitorId':fullVisit, 'PredictedLogRevenue':pred_y}
submit_df = pd.DataFrame(data)

In [48]:
submit_df.head()

,fullVisitorId,PredictedLogRevenue
0,7460955084541987166,-4.119009e-03
1,460252456180441002,-2.952371e-07
2,3461808543879602873,-3.444114e-07
3,975129477712150630,3.324347e-03
4,8381672768065729990,-6.881334e-03


In [49]:
type(submit_df.fullVisitorId[0])

str

In [50]:
submit_df.fullVisitorId.value_counts()[:5]

650107116874202739     102
7706472452740899006     86
7282998257608986241     85
6246006502985590876     72
8801084265240272984     71
Name: fullVisitorId, dtype: int64

In [51]:
submit_df2 = submit_df.drop_duplicates(['fullVisitorId'], keep='last')

In [52]:
submit_df2.shape

(296723, 2)

In [53]:
stand_submit_df = pd.read_csv(r'D:\Data\kaggle\GooAvePre\11.19\sample_submission_v2.csv')

C:\Users\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
stand_submit_df.shape

(296530, 2)

In [55]:
stand_submit_df.head()

,fullVisitorId,PredictedLogRevenue
0,0000018966949534117,0.0
1,0000039738481224681,0.0
2,0000073585230191399,0.0
3,0000087588448856385,0.0
4,0000149787903119437,0.0


In [56]:
type(stand_submit_df.fullVisitorId[1])

str

In [57]:
submit_df2.head()

,fullVisitorId,PredictedLogRevenue
4,8381672768065729990,-6.881334e-03
8,459669224143241747,-7.949592e-03
9,589440789980446309,-2.576150e-07
10,5299820794845174508,-3.118046e-07
11,7414906446453226824,-2.811525e-07


In [58]:
type(submit_df2.fullVisitorId[4])

str

In [59]:
submit_df2.fullVisitorId = submit_df2.fullVisitorId.str.rjust(19, '0')

C:\Users\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [60]:
submit_df2.head()

,fullVisitorId,PredictedLogRevenue
4,8381672768065729990,-6.881334e-03
8,0459669224143241747,-7.949592e-03
9,0589440789980446309,-2.576150e-07
10,5299820794845174508,-3.118046e-07
11,7414906446453226824,-2.811525e-07


In [61]:
print(stand_submit_df.shape)
print(submit_df2.shape)

(296530, 2)
(296723, 2)


In [62]:
result = pd.merge(stand_submit_df, submit_df2, how='left', on='fullVisitorId')

In [63]:
result.head()

,fullVisitorId,PredictedLogRevenue_x,PredictedLogRevenue_y
0,0000018966949534117,0.0,-2.344163e-07
1,0000039738481224681,0.0,1.626064e-03
2,0000073585230191399,0.0,-8.020329e-03
3,0000087588448856385,0.0,-3.444114e-07
4,0000149787903119437,0.0,-5.292175e-03


In [64]:
result.drop('PredictedLogRevenue_x', axis = 1, inplace=True)

In [65]:
result = result.fillna(0.0)

In [66]:
result.head(10)

,fullVisitorId,PredictedLogRevenue_y
0,0000018966949534117,-2.344163e-07
1,0000039738481224681,1.626064e-03
2,0000073585230191399,-8.020329e-03
3,0000087588448856385,-3.444114e-07
4,0000149787903119437,-5.292175e-03
5,0000196310838896290,-2.344163e-07
6,00001995526696366,0.000000e+00
7,0000255704530917106,-2.495153e-07
8,0000268499301061358,-3.830760e-07
9,0000276747989270229,-2.952371e-07


In [67]:
result.shape

(296703, 2)

In [68]:
result.drop_duplicates(['fullVisitorId'], keep='last', inplace=True)

In [69]:
result.shape

(296530, 2)

In [70]:
result = result.rename(columns={'PredictedLogRevenue_y': 'PredictedLogRevenue'})

In [71]:
result.shape

(296530, 2)

In [72]:
result.to_csv(r'D:\Data\kaggle\GooAvePre\11.23\submission.csv', index=False, encoding='utf-8')